In [ ]:
# import and mount drive to get acess to the files there
from google.colab import drive
drive.mount('/content/gdrive')

Primero, debe obtener las coordenadas de los 68 puntos característicos de la cara y escribirlas en un archivo txt para su uso posterior. El código utilizado aquí

In [4]:
import dlib
import cv2

predictor_path  = "/content/gdrive/MyDrive/Colab Notebooks/shape_predictor_68_face_landmarks.dat"
png_path = "/content/gdrive/MyDrive/images/GSA.jpg"

txt_path = "/content/gdrive/MyDrive/Colab Notebooks/points.txt"
f = open(txt_path,'w+')


detector = dlib.get_frontal_face_detector()
predicator = dlib.shape_predictor(predictor_path)
win = dlib.image_window()
img1 = cv2.imread(png_path)


dets = detector(img1,1)
print("Number of faces detected : {}".format(len(dets)))
for k,d in enumerate(dets):
    print("Detection {}  left:{}  Top: {} Right {}  Bottom {}".format(
        k,d.left(),d.top(),d.right(),d.bottom()
    ))
    lanmarks = [[p.x,p.y] for p in predicator(img1,d).parts()]
    for idx,point in enumerate(lanmarks):
        f.write(str(point[0]))
        f.write("\t")
        f.write(str(point[1]))
        f.write('\n')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


RuntimeError: ignored

2. Use el tamaño de la imagen para crear un rango rectangular (porque los puntos de características faciales están todos en la imagen), cree una instancia de Subdiv2D (esta clase se usará en el dibujo de las siguientes dos imágenes) e inserte todos los puntos en la clase creada:

In [ ]:
 #Create an instance of Subdiv2d
    subdiv = cv2.Subdiv2D(rect)
    #Create an array of points
    points = []
    #Read in the points from a text file
    with open("E:/data_ceshi/points.txt") as file:
        for line in file:
            x,y = line.split()
            points.append((int(x),int(y)))
    #Insert points into subdiv
    for p in points:
        subdiv.insert(p)


3. Dibuje la triangulación de Delaunay en la imagen original y obtenga una vista previa. Aquí agregué el efecto de animación dibujando por segmento de línea (usando el bucle for)

In [ ]:
#Draw delaunay triangles
def draw_delaunay(img,subdiv,delaunay_color):
    trangleList = subdiv.getTriangleList()
    size = img.shape
    r = (0,0,size[1],size[0])
    for t in  trangleList:
        pt1 = (t[0],t[1])
        pt2 = (t[2],t[3])
        pt3 = (t[4],t[5])
        if (rect_contains(r,pt1) and rect_contains(r,pt2) and rect_contains(r,pt3)):
            cv2.line(img,pt1,pt2,delaunay_color,1)
            cv2.line(img,pt2,pt3,delaunay_color,1)
            cv2.line(img,pt3,pt1,delaunay_color,1)
            
 #Insert points into subdiv
    for p in points:
        subdiv.insert(p)

        #Show animate
        if animate:
            img_copy = img_orig.copy()
            #Draw delaunay triangles
            draw_delaunay(img_copy,subdiv,(255,255,255))
            cv2.imshow(win_delaunary,img_copy)
            cv2.waitKey(100)


4. Finalmente dibuja el diagrama de Voronoi

In [ ]:
 def draw_voronoi(img,subdiv):
    (facets,centers) = subdiv.getVoronoiFacetList([])

    for i in range(0,len(facets)):
        ifacet_arr = []
        for f in facets[i]:
            ifacet_arr.append(f)

        ifacet = np.array(ifacet_arr,np.int)
        color = (random.randint(0,255),random.randint(0,255),random.randint(0,255))
        cv2.fillConvexPoly(img,ifacet,color)
        ifacets = np.array([ifacet])
        cv2.polylines(img,ifacets,True,(0,0,0),1)
        cv2.circle(img,(centers[i][0],centers[i][1]),3,(0,0,0))
    
  for p in points:
        draw_point(img,p,(0,0,255))

  #Allocate space for Voroni Diagram
  img_voronoi = np.zeros(img.shape,dtype = img.dtype)

  #Draw Voonoi diagram
  draw_voronoi(img_voronoi,subdiv)
